<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/FastAI_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 39.7 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 6.5 MB 34.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install fastai --upgrade

     |████████████████████████████████| 189 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from fastai.text.all import *
import torch
from torch import nn

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')
df = pd.read_csv('/content/drive/MyDrive/GHDomains/expanded_frame_final.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = df[['Description', 'Domain']]
df

,Description,Domain
0,freeCodeCamp.org's open source codebase and curriculum. Learn to code for free.,Documentation
1,"The most popular HTML, CSS, and JavaScript framework for developing responsive, mobile first projects on the web.",Web libraries and frameworks
2,"Bring data to life with SVG, Canvas and HTML. :bar_chart::chart_with_upwards_trend::tada:",Web libraries and frameworks
3,"A declarative, efficient, and flexible JavaScript library for building user interfaces.",Web libraries and frameworks
4,AngularJS - HTML enhanced for web apps!,Web libraries and frameworks
...,...,...
4995,This project aims to provide a working page flip implementation for usage in ListView.,Non-web libraries and frameworks
4996,JavaScript Client-Side Cookie Manipulation Library,Web libraries and frameworks
4997,The earliest versions of the very first c compiler known to exist in the wild written by the late legend himself dmr.,Software tools
4998,A clone of the UIImagePickerController using the Assets Library Framework allowing for multiple asset selection,Non-web libraries and frameworks


In [ ]:
labels = {'Application software':0,
          'Documentation':1,
          'Non-web libraries and frameworks':2,
          'Software tools':3,
          'System software':4,
          'Web libraries and frameworks':5
          }

df = df.replace({'Domain':labels})
df = df.rename(columns={"Description": "text", "Domain": "labels"})

In [ ]:
np.random.seed(112)
train_df, valid_df, test_df = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(train_df),len(valid_df), len(test_df))

4000 500 500


In [ ]:
hf_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
hf_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [ ]:
class HF_Dataset(torch.utils.data.Dataset):
    def __init__(self, df, hf_tokenizer):
        self.df = df
        self.hf_tokenizer = hf_tokenizer
        
        # label 1 is negative sentiment and label 2 is positive sentiment
        self.label_map = {0:0, 1:1,2:2,3:3,4:4,5:5}
        
    def __len__(self):
        return len(self.df)

    def decode(self, token_ids):
        return ' '.join([hf_tokenizer.decode(x) for x in tokenizer_outputs['input_ids']])
    
    def decode_to_original(self, token_ids):
        return self.hf_tokenizer.decode(token_ids.squeeze())

    def __getitem__(self, index):
        label, title, text = self.df.iloc[index]
        label = self.label_map[label]
        label = torch.tensor(label)

        tokenizer_output = self.hf_tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        
        tokenizer_output['input_ids'].squeeze_()
        tokenizer_output['attention_mask'].squeeze_()
        
        return tokenizer_output, label

In [ ]:
train_dataset = HF_Dataset(train_df, hf_tokenizer)
valid_dataset = HF_Dataset(valid_df, hf_tokenizer)
len(train_dataset), len(valid_dataset)

(4000, 500)

In [ ]:
train_dl = DataLoader(train_dataset, bs=16, shuffle=True)
valid_dl = DataLoader(valid_dataset, bs=16)
dls = DataLoaders(train_dl, valid_dl)

In [ ]:
batched_data, batched_labels = train_dl.one_batch()
batched_data.keys(), batched_data['input_ids'].shape, batched_labels.shape

ValueError: ignored

In [ ]:
hf_model(**batched_data)

In [ ]:
class HF_Model(nn.Module):
  
    def __init__(self, hf_model):
        super().__init__()
        
        self.hf_model = hf_model
        
    def forward(self, tokenizer_outputs):
        
        model_output = self.hf_model(**tokenizer_outputs)
        
        return model_output.logits

In [ ]:
model = HF_Model(hf_model)

In [ ]:
logits = model(batched_data)
logits

In [ ]:
# (doesn't automatically place model + data on gpu otherwise)
learn = Learner(dls, model.cuda(), loss_func=nn.CrossEntropyLoss(), metrics=[accuracy])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, 1e-4)
